In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from datetime import datetime
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer

import sys
sys.path.append("/home/kieran/code/Ant-mel/legendary_game_recs/")

from preprocessing.preprocess_1_cleaning import *
from preprocessing.preprocess_2_features import *
from preprocessing.pipeline_the_unification import *

In [4]:
raw_data = pd.read_csv("../raw_data/all_game_data_v1_corrected2.csv", low_memory=False)
topic_data = pd.read_csv("../raw_data/topics.csv", low_memory=False)

In [6]:
reference_data = pipeline_genre_ohe_only(raw_data)

/home/kieran/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column]=pd.to_datetime(df[column])
/home/kieran/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_2_features.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupe['gen'] = None
/home/kieran/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [7]:
topic_data

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
36815,0,0,0,0,1,0,0,0,0,0
36816,0,0,0,0,0,0,0,1,0,0
36817,0,0,0,1,0,0,0,0,0,0
36818,0,0,0,0,0,0,1,0,0,0


In [8]:
reference_data = pd.concat((reference_data,topic_data), axis=1)

In [10]:
reference_data.columns

Index(['title', 'release_date', 'plays', 'playing', 'backlogs', 'wishlist',
       'developers', 'avg_review', 'platforms', 'description', 'total_reviews',
       'total_lists', 'main', 'ratings_zero_five', 'ratings_one_zero',
       'ratings_one_five', 'ratings_two_zero', 'ratings_two_five',
       'ratings_three_zero', 'ratings_three_five', 'ratings_four_zero',
       'ratings_four_five', 'ratings_five_zero', 'image', 'url', 'Adventure',
       'Indie', 'RPG', 'Simulator', 'Puzzle', 'Shooter', 'Strategy',
       'Platform', 'Arcade', 'Sport', 'gen_1', 'gen_2', 'gen_3', '0', '1', '2',
       '3', '4', '5', '6', '7', '8', '9'],
      dtype='object')

In [11]:
processed_data = drop_unnecesary_coulumns(reference_data)

In [12]:
processed_data.head()

,plays,playing,backlogs,wishlist,avg_review,total_reviews,total_lists,Adventure,Indie,RPG,...,0,1,2,3,4,5,6,7,8,9
0,7,0,2,2,2.2,1,4,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,80,1,82,297,3.3,8,104,0,1,0,...,0,0,0,0,0,0,0,1,0,0
2,9,0,5,1,2.4,0,3,1,0,1,...,0,0,0,0,0,1,0,0,0,0
3,10,3,9,41,4.5,5,12,1,0,0,...,0,0,1,0,0,0,0,0,0,0
4,16,0,26,155,2.1,12,27,1,1,0,...,0,0,0,0,1,0,0,0,0,0


In [13]:
X_train = processed_data.drop(['avg_review'], axis=1)
y_train = processed_data[['avg_review']]

In [14]:
X_train.head()

,plays,playing,backlogs,wishlist,total_reviews,total_lists,Adventure,Indie,RPG,Simulator,...,0,1,2,3,4,5,6,7,8,9
0,7,0,2,2,1,4,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,80,1,82,297,8,104,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,9,0,5,1,0,3,1,0,1,1,...,0,0,0,0,0,1,0,0,0,0
3,10,3,9,41,5,12,1,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,16,0,26,155,12,27,1,1,0,0,...,0,0,0,0,1,0,0,0,0,0


In [15]:
y_train

,avg_review
0,2.2
1,3.3
2,2.4
3,4.5
4,2.1
...,...
36815,3.8
36816,2.7
36817,2.2
36818,2.0


In [16]:
num_transformer = Pipeline([('imputer', SimpleImputer(strategy="median")),
                             ('mm_scaler', MinMaxScaler())
                            ])

In [17]:
X_train_scaled = num_transformer.fit_transform(X_train)

In [18]:
pd.DataFrame(X_train_scaled)

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,0.000130,0.000161,0.000167,0.000313,0.000263,0.000690,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.001717,0.000323,0.006833,0.046406,0.002105,0.017931,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.000174,0.000161,0.000417,0.000156,0.000000,0.000517,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.000196,0.000645,0.000750,0.006406,0.001316,0.002069,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000326,0.000161,0.002167,0.024219,0.003158,0.004655,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36815,0.000065,0.000161,0.000083,0.000625,0.000000,0.000345,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
36816,0.000109,0.000161,0.000000,0.000156,0.000263,0.000345,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
36817,0.000043,0.000161,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
36818,0.000370,0.000161,0.000000,0.000000,0.000526,0.001207,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# Model and Finetuning

In [19]:
# Import the necessary libraries
from sklearn.inspection import permutation_importance
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier  # Import KNeighborsClassifier or KNeighborsRegressor as needed

# Fit a KNN model to your data
knn_model = KNeighborsRegressor(n_neighbors=11).fit(X_train_scaled, y_train)  # You can adjust the number of neighbors (k) as needed
# X and y are your feature matrix and target variable



In [47]:
# Perform permutation importance analysis
permutation_score = permutation_importance(knn_model, X_train_scaled, y_train, n_repeats=10)



In [48]:
# Create a DataFrame to store the results
importance_df = pd.DataFrame(np.vstack((X_train.columns, permutation_score.importances_mean)).T)
importance_df.columns = ['feature', 'score_decrease']

# Sort the DataFrame by feature importance in descending order
importance_df = importance_df.sort_values(by='score_decrease', ascending=False)

In [49]:
importance_df

,feature,score_decrease
3,wishlist,0.121758
5,total_lists,0.080465
6,Adventure,0.063987
2,backlogs,0.056123
7,Indie,0.054861
11,Shooter,0.038195
8,RPG,0.037922
4,total_reviews,0.035428
0,plays,0.032854
12,Strategy,0.031603


# Prediction

In [20]:
reference_data[reference_data['title'] == "The Legend of Zelda: Breath of the Wild"]

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,platforms,description,...,0,1,2,3,4,5,6,7,8,9
11702,The Legend of Zelda: Breath of the Wild,2017-03-03,42000,3500,6600,3400,"[Nintendo EPD Production Group No. 3, Nintendo]",4.4,"[Wii U, Nintendo Switch]",The Legend of Zelda: Breath of the Wild is the...,...,0,0,0,0,0,0,0,1,0,0


In [68]:
reference_data[reference_data['title'] == "The Sims 4"]

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,platforms,description,...,0,1,2,3,4,5,6,7,8,9
16006,The Sims 4,2014-09-02,12000,640,1000,136,"[Electronic Arts, The Sims Studio]",3.1,"[Windows PC, Mac, PlayStation 4, Xbox One, Pla...",Unleash your imagination and create a unique w...,...,0,0,0,0,0,1,0,0,0,0


In [73]:
reference_data[reference_data['title'] == "Pokémon Go"]

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,platforms,description,...,0,1,2,3,4,5,6,7,8,9
12935,Pokémon Go,2016-07-06,17000,746,267,24,"[The Pokémon Company, Niantic Labs]",2.7,"[Android, iOS]",Travel between the real world and the virtual ...,...,0,0,0,0,0,1,0,0,0,0


In [78]:
reference_data[reference_data['title'] == "Tetris"]

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,platforms,description,...,0,1,2,3,4,5,6,7,8,9
32736,Tetris,1994-12-31,17,0,0,1,[Nokia],2.5,[Mobile],A Tetris clone released built-in the Hagenuk M...,...,0,0,1,0,0,0,0,0,0,0
36238,Tetris,1984-06-06,112,0,20,12,[Alexey Pajitnov],4.1,[Legacy Computer],The original version of Tetris was created by ...,...,0,0,0,0,0,0,0,1,0,0


In [59]:
pikmin_array = X_train_scaled[76:77]

In [79]:
tetris_array = X_train_scaled[36238:36289]

In [64]:
wow_array = X_train_scaled[26212:26213]

In [25]:
border_array = X_train_scaled[7157:7158]

In [26]:
wipeout_array = X_train_scaled[32334:32335]

In [43]:
dk_array = X_train_scaled[32871:32872]

In [54]:
dungeon_keeper = X_train_scaled[31397:31398]

In [69]:
sims4_array = X_train_scaled[16006:16007]

In [74]:
pokemongo_array = X_train_scaled[12935:12936]

In [27]:
 array_inputs = np.concatenate((border_array, zelda_array))

In [80]:
ind_list = list(knn_model.kneighbors(tetris_array,n_neighbors=5000)[1][0])

In [81]:
output_df = reference_data.iloc[ind_list, :]

In [82]:
output_df.head(15)

,title,release_date,plays,playing,backlogs,wishlist,developers,avg_review,platforms,description,...,0,1,2,3,4,5,6,7,8,9
36238,Tetris,1984-06-06,112,0,20,12,[Alexey Pajitnov],4.1,[Legacy Computer],The original version of Tetris was created by ...,...,0,0,0,0,0,0,0,1,0,0
35155,Plotting,1989-12-31,10,0,3,2,[Taito Corporation],2.4,[Arcade],The screen is broken into two halves where eac...,...,0,0,0,0,0,0,0,1,0,0
35454,Titan,1988-12-31,8,0,7,1,"[Fujitsu Interactive, Titus France]",3.1,"[FM Towns, Atari ST/STE, Amiga, PC DOS, Family...",Titan takes the Breakout concept and adds anot...,...,0,0,0,0,0,0,0,1,0,0
36753,Avalanche,1978-04-01,24,0,2,1,[Atari],2.1,[Arcade],Avalanche is a 2-D game of skill. You control ...,...,0,0,0,0,0,0,0,1,0,0
36776,Amazing Maze,1976-10-01,6,0,1,0,"[Dave Nutting Associates, Midway Manufacturing]",1.6,[Arcade],The Amazing Maze Game is an arcade game develo...,...,0,0,0,0,0,0,0,1,0,0
35827,Wonder Momo,1987-02-24,43,0,28,19,"[Bandai Namco Games, Namco]",2.2,"[Arcade, Wii, Mobile, TurboGrafx-16/PC Engine]",Wonder Momo is a loose parody of a typical Ult...,...,0,0,0,0,0,0,0,1,0,0
34041,World Heroes,1992-07-28,157,0,22,23,"[Playtronic, Alpha Denshi]",2.5,"[Neo Geo CD, PlayStation 3, Sega Mega Drive/Ge...",Are you ready for a fierce battle in an ultra-...,...,0,0,0,0,0,0,0,1,0,0
33448,Power Instinct,1993-11-01,56,0,16,17,"[Atlus, Atlus USA]",2.8,[Arcade],The original Arcade version of Power Instinct....,...,0,0,0,0,0,0,0,1,0,0
34957,Pit-Fighter,1990-08-01,159,0,22,2,"[Tec Toy, Teque London]",1.5,"[Sega Mega Drive/Genesis, Atari ST/STE, Arcade...",The gameplay is similar to Taito's Violence Fi...,...,0,0,0,0,0,0,0,1,0,0
36510,Super Pac-Man,1982-09-26,203,0,24,11,"[Namco, Bally Midway]",2.8,"[Arcade, Mobile, PC DOS, Commodore C64/128]",Super Pac-Man is a game in Namco's Pac-Man fam...,...,0,0,0,0,0,0,0,1,0,0


In [ ]:
year_limit = '1997'

In [ ]:
rslt_df = output_df[output_df['release_date'] <= '1997-01-01']

In [ ]:
rslt_df.head(20)